In [15]:
import os
from lxml import html
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import os
import pandas as pd
from bs4 import BeautifulSoup

In [16]:
# Function to extract reels data from HTML content
def extract_text_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all reels data blocks
    reel_blocks = soup.find_all('div', class_='pam _3-95 _2ph- _a6-g uiBoxWhite noborder')
    
    reels_data = []
    
    for block in reel_blocks:
        reel_info = {}
        
        # Extract video source
        video = block.find('video')
        if video and video.get('src'):
            reel_info['Video Source'] = video['src']
        
        # Extract caption
        caption_div = block.find('td', class_='_2pin _a6_q', colspan='2')
        if caption_div:
            caption_text = caption_div.find_all('div')[-1].get_text(strip=True)
            reel_info['Caption'] = caption_text
        
        # Extract upload timestamp
        timestamp_td = block.find('td', class_='_2pin _2piu _a6_r')
        if timestamp_td:
            reel_info['Upload Timestamp'] = timestamp_td.get_text(strip=True)
        
        reels_data.append(reel_info)
    
    return reels_data




In [ ]:
# Function to extract numeric data from HTML content
def extract_numeric_info(html_content, post_number, column_number):
    # Parse the HTML content
    tree = html.fromstring(html_content)
    
    # Use the specific XPath
    xpath = f'/html/body/div/div/div/div[2]/div[2]/div[{post_number}]/div/table[2]/tbody/tr[{column_number}]/td'

    # Find the element
    elements = tree.xpath(xpath)
    
    # Extract and process the text
    data = []
    for element in elements:
        # Get the text of the element, stripping whitespace
        text = element.text_content().strip()
        data.append(text)
    return data

In [ ]:
myFile=open('reels.html','r')
soup=BeautifulSoup(myFile,"html5lib")

html_content = soup.prettify()

# Extract reels text data
reels = extract_text_info(html_content)

# Convert to DataFrame
df = pd.DataFrame(reels)

total_posts = df.shape[0] - 1

# Pass first 2 columns to get the column names
for column_number in range(3, 10):
        scraped_data = extract_numeric_info(html_content, 1, column_number)
        column_name=scraped_data[0].replace('\n', '').rsplit(' ', 1)[0].replace(' ', '')
        info = scraped_data[0].replace('\n', '').rsplit(' ', 1)[1].replace(' ', '')
        
        df[column_name] = int(info)


# Extract numeric data for each post
for post_number in range(1, total_posts):
    for column_number in range(3, 10):
        scraped_data = extract_numeric_info(html_content, post_number, column_number)
        column_name=scraped_data[0].replace('\n', '').rsplit(' ', 1)[0].replace(' ', '')
        info = scraped_data[0].replace('\n', '').rsplit(' ', 1)[1].replace(' ', '')
        
        df.loc[post_number-1, column_name] = int(info)


df.to_csv('reels_data.csv', index=False, encoding='utf-8')
